In [ ]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px


def filter_data(df,tech,sector):
    
    df = df[df["NAICS2017_LABEL"] == sector]

    df = df[df["FACTORS_P_LABEL"].str.contains(tech)]

    df = df[df["FIRMPDEMP_PCT"] > 0]

    return df

df = pd.read_excel("TechImpactCensusData.xlsx",engine='openpyxl')

df.drop(columns=["SEX_LABEL","ETH_GROUP_LABEL","RACE_GROUP_LABEL","VET_GROUP_LABEL","NSFSZFI_LABEL"],inplace= True)

df = df[df["NAICS2017_LABEL"] != "Industries not classified"]
df.drop(df.index[df["FACTORS_P"].str.contains("99")],inplace = True)
df.drop(df.index[df["FACTORS_P"].str.contains("10")],inplace = True)
df.drop(df.index[df["FACTORS_P"].str.contains("09")],inplace = True)

sectors_list = []

for x in df["NAICS2017_LABEL"].values:

    sectors_list.append(x)

sectors_list = list(set(sectors_list))

XLRDError: Excel xlsx file; not supported

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("Sectors:"),
    dcc.Dropdown(
        id='sectors',
        options=[{'value': x, 'label': x} 
                 for x in sectors_list],
        clearable=False
    ),
    html.P("Tech:"),
    dcc.Dropdown(
        id = "tech",
        options=[{"value" : x, "label": x} for x in ["Artificial Intelligence","Cloud-Based","Specialized Software","Robotics","Specialized Equipment"]],
        clearable=False
    ),
    dcc.Graph(id="chart"),
])

@app.callback(
    Output("chart", "figure"),
    [Input("sectors", "value"),
     Input("tech", "value")])
     
def generate_chart(sector,Tech):
    
    return px.pie(filter_data(df,Tech,sector),values = "FIRMPDEMP_PCT",names = "FACTORS_P_LABEL")
    

app.run_server()